Constant-Q를 이용한 머신러닝 오디오 분류

데이터 준비
   - wav는 매 순간의 음압을 측정하여 그 수치를 저장한 형태이기 때문에 그 자체로 음악을 분석하기에 적합하지 않음(음의 높이와 세기를 듣는것이지 순간의 음압을 듣는게 아니기 때문)
   - 푸리에 변환과 같은 변환 기법을 이용하여 시간 축의 데이터를 주파수 축의 데이터로 바꿔줘야할 필요가 있음
   - 푸리에 변환 대신 푸리에 변환과 유사한 Constant-Q 변환을 사용
   - Constant-Q 변환은 주파수 축이 로그 단위로 변환되고, 각 주파수에 따라 해상도가 다양하게 처리되기 때문에(저주파는 저해상도, 고주파는 고해상도) 음악을 처리하는 데에 푸리에 변환보다 유리
   - 주파수 대역을 저장할 리스트 audio_cqt 선언
   - constant-Q 변환할 때는 변환할 오디오 데이터와 sampling rate가 필요
   - 해당 데이터에서는 sampling rate가 모두 동일하므로 따로 처리가 필요하지 않음
   - 여기서는 Constant-Q 변환을 사용해 오디오 데이터를 주파수 대역으로 변환

   - 변환에는 앞서 준비한 데이터를 가져와 사용하며, Constant-Q 변환에는 librosa.cqt 함수를 사용
   - 여기서 n_bins는 옥타브 단계 및 개수를, bins_per_octave는 한 옥타브가 가지는 단계를 의미
   - 라벨에 대해선 원 핫 인코딩을 적용

In [ ]:
import numpy as np
import itertools
import librosa
import librosa.display 
import IPython.display as ipd
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

In [ ]:
midi_file = './GeneralMidi.wav'
instruments = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
num_notes = 50
sec = 2

audio = []
inst = []
for inst_idx, note in itertools.product(range(len(instruments)), range(num_notes)):
    instrument = instruments[inst_idx]
    offset = (instrument*num_notes*sec + (note*sec))
    print('instrument : {}, note: {}, offset : {}'.format(instrument, note, offset))
    y, sr = librosa.load(midi_file, sr=None, offset=offset, duration=2.0)
    audio.append(y)
    inst.append(inst_idx)

In [ ]:
audio_cqt = []
for y in audio:
    ret = librosa.cqt(y, sr, hop_length=1024, n_bins=24*7, bins_per_octave=24)
    ret = np.abs(ret)

   - 앞서 생성한 주파수 대역을 spectrogram을 시각화
   - 악기 간 spectrogram을 비교해보면 차이가 존재함을 알 수 있음

In [ ]:
for i in range(0, len(instruments)*num_notes, num_notes):
    amp_db = librosa.amplitude_to_db(np.abs(audio_cqt[i]), ref=np.max)
    librosa.display.specshow(amp_db, sr=sr, x_axis='time', y_axis='cqt_note')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Instrument : {}'.format(inst[i]))
    plt.tight_layout()
    plt.show()

   - 훈련 데이터와 실험 데이터 분리

In [ ]:
cqt_np = np.array(audio_cqt, np.float32)
inst_np = np.array(inst, np.int16)

print(cqt_np.shape, inst_np.shape))

   - 분류기에서 사용하기 위해 3차원 벡터를 2차원 벡털 변환

In [1]:
cqt_np = cqt_np.reshape((500, 168 * 87))

NameError: name 'cqt_np' is not defined

   - 읽어온 데이터는 음량이나 범위가 다를 수 있음
   - min-max scaling을 통해 데이터의 범위를 조정

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(cqt_np)

   - 학습 데이터와 실험 데이터를 분리

In [ ]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(cqt_np, inst_np, test_size=0.2)

print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)

Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

LR = LogisticRegression()
LR.fit(train_x, train_y)
pred = LR.predict(test_x)
acc = accuracy_score(pred, test_y)
print(accuracy_score)

Support Vector Machine

In [ ]:
from sklearn import svm

SVM = svm.SVC(kernel='linear')
SVM.fit(train_x, train_y)
pred = SVM.predict(test_x)
acc = accuracy_score(pred, test_y)
print(acc)

Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(train_x, train_y)
pred = DT.predict(test_x)
acc = accuracy_score(pred, test_y)
print(acc)